<h1><center>Run a Kafka Producer of clickstream events in Message Hub </center></h1>

## Introduction
The Apache Kafka Producer is an IBM WebSphere Liberty Java EE application that generates a stream of clickstream events to simulate user behavior on a web site. For example, log in, browse, add to cart, purchase, and log out events are all simulated. 

There are two ways to deploy the Kafka Producer:
- Deploy the Kafka Producer in Bluemix Liberty runtime environment. If you want to make changes to the Kafka Producer, the notebook guides you how to customize and rebuild the Kafka Producer code.
- Run the Kafka Producer directly from the notebook.

The figure below shows the flow from the Jupyter notebook or a Bluemix Liberty environment, to a Kafka Producer, to a Message Hub, and to a streaming pipeline in [Notebook #2: Creating a streaming pipeline](https://apsportal.ibm.com/analytics/notebooks/d06e9e69-f8c9-4608-9e07-deb10fc4f85f/view?projectid=81238e6c-a19b-4c5c-9e45-753dfe7b7de3&context=analytics). 

<img src='https://github.com/notebookgraphics/advobeta/blob/master/NB1_FLOW_OVERVIEW_sm.gif?raw=true'></img>

The Kafka Producer code can be found at [advo-beta-producer](https://github.com/ibm-cds-labs/advo-beta-producer).

This notebook runs on Python 2 with Spark 2.0. 

## Table of contents
1. [Setup](#setup)<br>

2. [Deploy a Kafka Producer in a Bluemix Liberty runtime environment](#deploy)<br>

3. [(Optional) Customize and rebuild the Kafka Producer code](#customize)<br>

4. [Run the Kafka Producer directly from a Jupyter notebook](#run_direct)<br>
    4.1 [Import the Kafka Producer jar](#run_direct_step_1)<br>
    4.2 [Set the Message Hub credentials](#run_direct_step_2)<br>
    4.3 [Start the DataStream](#run_direct_step_3)<br>
    4.4 [Verify that events are flowing](#run_direct_step_4)<br>

5. [Summary and next steps](#summary)<br>
    

<a id="setup"></a>
## Setup
1. [Create an account in Bluemix](https://console.bluemix.net/registration/) and log in.
2. Create a space in your Bluemix account for your notebook work. For example, you might call the space *AdvoBeta*.
3. Create a Message Hub service instance by doing these steps: 
    - Click the **Menu** icon in the left of the task bar.
    - Click  **Services**, and then click **Dashboard**.
    - Click **Create Service**. Note the values of the following  Message Hub service credentials: *user*, *password*, and *api_key*.
5. [Download the CloudFoundry CLI](https://github.com/cloudfoundry/cli#downloads) on your local machine. This app is needed to deploy the Kafka Producer.
6. [Install git](https://git-scm.com/downloads) CLI.
6. (Optional) Download the [Eclipse IDE](https://developer.ibm.com/wasdev/downloads/#asset/tools-IBM_Eclipse_Tools_for_Bluemix) and install the Liberty development tooling. 

<a id="deploy"></a>
## Deploy a Kafka Producer in a Bluemix Liberty runtime environment

1. Clone the repository by running this command: `git clone https://github.com/ibm-watson-data-lab/advo-beta-producer.git`
2. [Connect to Bluemix by using the CloudFoundry CLI](https://console.bluemix.net/docs/runtimes/liberty/getting-started.html#getting-started-with-liberty-on-bluemix). To select the space and org that you created in the Setup section above, see step 4 in [Getting started with Liberty in Bluemix](https://console.bluemix.net/docs/runtimes/liberty/getting-started.html#getting-started-with-liberty-on-bluemix). Do these steps at a command line: 
    - `cf api <API_endpoint>` - where `<API_endpoint>` is the API region that you selected in the Setup section. For example, `cf api  https://api.ng.bluemix.net.`
    - `cf login` - to log in to your Bluemix account. For example, `cf login my_login@aaa.com`
    - From the list of organizations, select the organization. The "Targeted organization" and "Targeted space" are what you need.

3. Customize the Cloud Foundry manifest by doing these steps:
  - Open the file [`manifest.yml`](https://github.com/ibm-watson-data-lab/advo-beta-producer/blob/master/manifest.yml) in your favorite text editor. 
  - In the `env:` section, assign values for the following three variables (from the credentials in the [Setup](#setup) section, step 4): 
    - `KAFKA_USER_NAME` : MessageHub `user` name from service credentials  
    - `KAFKA_PASSWORD` : MessageHub `password` from service credentials  
    - `KAFKA_API_KEY` : MessageHub `api_key` from service credentials  
  - Set the `host` variable to what you want your route (subdomain) to be. The URL will be https://<host>.mybluemix.net/LocalCartKafkaProducer/
4. Deploy Bluemix by using the following command: `cf push -f manifest.yml -p ./dist/kafkaProducerServer.zip`
5. In a web browser, navigate to the app URL, for example https://<host>.mybluemix.net/LocalCartKafkaProducer/. The app should show you the number of events that are being generated. You can configure the generated rate of events to be between 0 and 20 events per second. **Note:** Choosing 0 events will pause the Kafka Producer.

<a id="customize"></a>
## (Optional) Customize and rebuild the Kafka Producer code 

1. If Eclipse is not installed, install [Eclipse Neon for Java EE Developers (4.6.1)](http://www.eclipse.org/downloads/eclipse-packages/).

2. Click and hold this <a href="http://marketplace.eclipse.org/marketplace-client-intro?mpc_install=1774120" target="_blank"><img src="https://console.bluemix.net/docs/api/content/starters/images/installbutton.png?lang=en" style="display:inline-block" alt="Drag and drop into a running Eclipse Neon workspace to install IBM Eclipse Tools for Bluemix"></a> button to drag and drop it into the Eclipse toolbar. Follow the prompts to install IBM Eclipse Tools for Bluemix.

3. Import the advo-beta-producer project that you cloned in step 1 of the [Deploy a Kafka Producer](#deploy) section.

4. Become familiar with the Producer code:  

    a. **com.ibm.localcart.DataStream:** Singleton class responsible for generating the Kafka events. It spawns a thread that executes every second and generate new events from the master [JSON file](https://github.com/ibm-watson-data-lab/advo-beta/raw/master/data/dataStream.json) on github. The number of events generated each second is configurable from the UI. The default is 2.
    
    b. **com.ibm.localcart.MessageHubConfig:** Singleton class responsible for configuring the Kafka Producer.  
    
    c. **com.ibm.localcart.StatsAPI:** Jax-rs application class that provide rest end-points like *getStats* and *getConfig* that are used by the UI.
    
    d. **/src/main/webapp/index.html:** Main UI that provide statistics for each topic generated. It also allows you to change the number of events that are being generated from 0 - 20. To pause the Producer, set the slider to 0.  
    
    e. **/src/main/wlp/server.xml:** Server configuration. It contains server deployment information. Take a minute to look at the jaasLoginModule section that configures jaas security for Message Hub. It uses the KAFKA_PASSWORD and KAFKA_USER_NAME variables that can be passed in server.env
    
    f. Optional: As an exercise, change how events are generated to use a different URL than the github URL that is currently used. You could also use a more realistic scenario where the events are read from a data source such as a relational database.  
    
    g. Because we need the custom jaasLoginModule secton in server.xml, we need to create a new zip file in order to deploy on Bluemix. For instructions about how to generate the zip file, see [Custom Liberty server.xml configurations in IBM Bluemix](https://www.ibm.com/blogs/bluemix/2015/01/modify-liberty-server-xml-configurations-ibm-bluemix/).  
    
    h. Deploy the zip from a command line terminal by using the following command: `cf push LocalCartKafkaProducer -p ./kafkaProducerServer.zip`  

<a id="run_direct"></a>
## Run the Kafka Producer directly from a notebook
If you want to get started quickly and generate events, you can simply run the following cells directly from your notebook. 

The drawback is that the event generation depends on the notebook session being kept alive. When the session stops, event generation also stops.

The code below uses the PixieDust PackageManager to load the Kafka Producer code from a jar file. It then uses the Scala bridge to directly invoke the DataStream class and start producing events.

<a id="run_direct_step_1"></a>
### 1. Import the Kafka Producer jar by using the PixieDust PackageManager

In [9]:
import pixiedust
pixiedust.installPackage("https://github.com/ibm-watson-data-lab/advo-beta-producer/raw/master/dist/LocalCartKafkaProducer-1.0-SNAPSHOT-uber.jar")

Package already installed: https://github.com/ibm-watson-data-lab/advo-beta-producer/raw/master/dist/LocalCartKafkaProducer-1.0-SNAPSHOT-uber.jar


<a id="run_direct_step_2"></a>
### 2. Set the Message Hub credentials

In [10]:
%%scala
System.setProperty("KAFKA_USER_NAME","nRaQ5jRnWEX0ji8U")
System.setProperty("KAFKA_PASSWORD","6Ak8w9AryhAOjsTRYkY2oEHgljL1ZT0N")
System.setProperty("KAFKA_API_KEY","nRaQ5jRnWEX0ji8U6Ak8w9AryhAOjsTRYkY2oEHgljL1ZT0N")
System.setProperty("USE_JAAS", "true")

<a id="run_direct_step_3"></a>
### 3. Start the data stream

In [11]:
%%scala
import com.ibm.localcart.DataStream
println(DataStream.getInstance());

com.ibm.localcart.DataStream@14c94b9e


<a id="run_direct_step_4"></a>
### 4. Verify that events are flowing by looking at the stats
> You can run this cell multiple times to make sure that events continue to be generated.

In [ ]:
%%scala
import com.ibm.localcart.DataStream
println(DataStream.getInstance().getStats)

<a id="summary"></a>
## Summary and next steps
In this notebook you set up a stream of data that includes clickstream events (log in, browse, add to cart, logout without purchase, logout with purchase, and checkout). Use this data in notebook [2 - Creating a streaming pipeline](https://apsportal.ibm.com/analytics/notebooks/d06e9e69-f8c9-4608-9e07-deb10fc4f85f/view?projectid=81238e6c-a19b-4c5c-9e45-753dfe7b7de3&context=analytics).

### Author
David Taieb is a Distinguished Engineer for Watson Data Platform and a Developer Advocate at IBM. 

***

Copyright © IBM Corp. 2017. This notebook and its source code are released under the terms of the MIT License.